In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.linear_model import BayesianRidge
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import (SimpleImputer, KNNImputer)
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Ridge
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.multiclass import OneVsRestClassifier
scaler = MinMaxScaler()
np.set_printoptions(precision = 6)

### Data

In [2]:
df_train_features = pd.read_csv('train_features.csv', index_col=0)
df_train_labels   = pd.read_csv('train_labels.csv', index_col=0)
df_test_features  = pd.read_csv('test_features.csv', index_col=0)

display(df_train_features.head(10))
display(df_train_labels.head(10))
display(df_test_features.head(10))

X = df_train_features.values[:,1:]
y = df_train_labels.values
X_test = df_test_features.values[:,1:]

feature_index = df_train_features.index
label_index = df_train_labels.index
feature_index_test = df_test_features.index


,Time,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,BaseExcess,...,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH
pid,,,,,,,,,,,,,,,,,,,,,
1,3,34.0,NaN,NaN,12.0,NaN,36.0,8.7,24.0,-2.0,...,NaN,100.0,NaN,114.0,24.6,94.0,NaN,NaN,142.0,7.33
1,4,34.0,NaN,NaN,NaN,NaN,36.0,NaN,NaN,-2.0,...,NaN,100.0,NaN,NaN,NaN,99.0,NaN,NaN,125.0,7.33
1,5,34.0,NaN,NaN,NaN,NaN,36.0,NaN,NaN,0.0,...,NaN,100.0,NaN,NaN,NaN,92.0,NaN,NaN,110.0,7.37
1,6,34.0,NaN,NaN,NaN,NaN,37.0,NaN,NaN,0.0,...,NaN,100.0,NaN,NaN,NaN,88.0,NaN,NaN,104.0,7.37
1,7,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,100.0,NaN,NaN,22.4,81.0,NaN,NaN,100.0,7.41
1,8,34.0,NaN,NaN,NaN,NaN,37.0,NaN,NaN,NaN,...,NaN,100.0,NaN,NaN,22.4,82.0,NaN,NaN,106.0,NaN
1,9,34.0,NaN,NaN,NaN,NaN,37.0,NaN,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,67.0,NaN,NaN,112.0,NaN
1,10,34.0,NaN,NaN,NaN,NaN,37.0,NaN,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,62.0,NaN,NaN,121.0,NaN
1,11,34.0,NaN,NaN,12.0,NaN,NaN,8.5,26.0,NaN,...,NaN,100.0,NaN,111.0,23.8,58.0,NaN,NaN,118.0,NaN


,LABEL_BaseExcess,LABEL_Fibrinogen,LABEL_AST,LABEL_Alkalinephos,LABEL_Bilirubin_total,LABEL_Lactate,LABEL_TroponinI,LABEL_SaO2,LABEL_Bilirubin_direct,LABEL_EtCO2,LABEL_Sepsis,LABEL_RRate,LABEL_ABPm,LABEL_SpO2,LABEL_Heartrate
pid,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,12.1,85.4,100.0,59.9
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.8,100.6,95.5,85.5
100,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,16.5,88.3,96.5,108.1
1000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,19.4,77.2,98.3,80.9
10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.6,76.8,97.7,95.3
10002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.5,67.4,99.1,64.6
10006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.2,84.9,96.8,90.5
10007,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,23.8,97.0,94.3,76.0
10009,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,18.9,72.0,95.3,91.5


,Time,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,BaseExcess,...,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH
pid,,,,,,,,,,,,,,,,,,,,,
0,1,39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2,39.0,NaN,44.2,17.0,NaN,36.0,10.2,13.0,NaN,...,119.0,100.0,NaN,98.0,31.0,82.0,21.8,NaN,119.0,NaN
0,3,39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9.0,...,NaN,100.0,NaN,NaN,NaN,78.0,NaN,NaN,125.0,7.34
0,4,39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,80.0,NaN,NaN,136.0,NaN
0,5,39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,83.0,NaN,NaN,135.0,NaN
0,6,39.0,NaN,NaN,NaN,NaN,36.0,NaN,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,88.0,NaN,NaN,144.0,NaN
0,7,39.0,NaN,38.5,20.0,NaN,NaN,9.1,16.0,NaN,...,109.0,100.0,NaN,102.0,25.9,NaN,26.4,NaN,NaN,NaN
0,8,39.0,NaN,NaN,NaN,NaN,36.0,NaN,NaN,-4.0,...,NaN,100.0,NaN,NaN,NaN,90.0,NaN,NaN,129.0,7.40
0,9,39.0,NaN,NaN,NaN,NaN,36.0,NaN,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,90.0,NaN,NaN,121.0,NaN


In [3]:
pop_mean = np.nanmean(X, axis = 0)
pop_median = np.nanmedian(X, axis = 0)
pop_std = np.nanstd(X, axis = 0)
pop_max=np.max(X, axis = 0)
pop_mean=np.min(X, axis = 0)
pop_quantile_25=np.quantile(X,0.25,axis=0)
pop_quantile_75=np.quantile(X,0.75,axis=0)


pop_mean_test = np.nanmean(X_test, axis = 0)
pop_median_test = np.nanmedian(X_test, axis = 0)
pop_std_test = np.nanstd(X_test, axis = 0)

# Extract unique PIDs creating a dictionary
feature_index_unique = pd.unique(feature_index) # feature indexes
label_index_unique = pd.unique(label_index) # label indexes
feature_index_test_unique = pd.unique(feature_index_test) # feature indexes


In [4]:
def impute_approach(nb_Nan,col_median):
    if nb_Nan >= 12:
    # Estimate the score after replacing missing values by 0
        imputer = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=col_median)
    
    elif (nb_Nan>=5) and (nb_Nan<12):
    # Estimate the score after imputation (mean strategy) of the missing values
        imputer = SimpleImputer(missing_values=np.nan, strategy='median', fill_value=None)
    
    elif (nb_Nan < 5) and (nb_Nan>0):
    # Estimate the score after iterative imputation of the missing values
        imputer = IterativeImputer(estimator=BayesianRidge(), missing_values=np.nan, max_iter=10, tol=0.001, n_nearest_features=5, initial_strategy='mean')
    
    return imputer


In [5]:
def get_patient_data(X,pid_index):
    return X[pid_index*12:pid_index*12 + 12, :]

def get_patient_data_test(X,pid_index):
    return X[pid_index*12:pid_index*12 + 12, :]

In [6]:
x_imputed = np.zeros_like(X)
for pid_idx in range(len(feature_index_unique)):
    # get training data for a patient (dim 12 x d)
    x_patient = get_patient_data(X,pid_idx)
    x_patient_imputed = np.zeros_like(x_patient) # initialise imputed array for each patient
    
    
    for col in range(len(x_patient[1, :])):
        nan_count = np.isnan(x_patient[:, col]).sum()
        
        if nan_count>0:
            x_patient_col = x_patient[:, col].reshape(-1, 1) 
            #impute_estimator = impute_data(strategy, pop_median[col])
            impute_estimator=impute_approach(nan_count,pop_median[col])
            x_patient_imputed[:, col] = impute_estimator.fit_transform(x_patient_col).reshape(-1) # reshape to be size (12,)
        else:
            x_patient_imputed[:, col] = x_patient[:, col]           
    # Concatenate imputed data for every patient
    x_imputed[pid_idx*12:pid_idx*12 + 12, :] = x_patient_imputed

print("done")


x_test_imputed = np.zeros_like(X_test)
for pid_idx in range(len(feature_index_test_unique)):
    # get training data for a patient (dim 12 x d)
    x_test_patient = get_patient_data_test(X_test,pid_idx)
    x_test_patient_imputed = np.zeros_like(x_test_patient)
    
    # deal with column seperately
    for col in range(len(x_test_patient[1, :])):
        # Count number of Nan in each column
        nan_count = np.isnan(x_test_patient[:, col]).sum()
        
        if nan_count>0:
            x_test_patient_col = x_test_patient[:, col].reshape(-1, 1) 
            impute_estimator=impute_approach(nan_count,pop_median_test[col])
            x_test_patient_imputed[:, col] = impute_estimator.fit_transform(x_test_patient_col).reshape(-1) 
        else:
            x_test_patient_imputed[:, col] = x_test_patient[:, col]           
    x_test_imputed[pid_idx*12:pid_idx*12 + 12, :] = x_test_patient_imputed

print("done")

np.save('x_imputed3.npy', x_imputed)
np.save('x_test_imputed3.npy', x_test_imputed)

done
done


In [7]:
x_imputed = np.zeros((int(X.shape[0]/12), 35))

for pid_idx in range(len(feature_index_unique)):
    # get training data for a patient (dim 12 x d)
    x_patient = get_patient_data(X,pid_idx)
    x_patient_imputed = np.array([0.]*35)
    
    # deal with column seperately
    for col in range(len(x_patient[1, :])):
        # Count number of Nan in each column
        nan_count = np.isnan(x_patient[:, col]).sum()
        
        if nan_count<12:
            x_patient_imputed[col]=np.nanmean(x_patient, axis = 0)[col]
        else:
            x_patient_imputed[col] = -1           
    # Concatenate imputed data for every patient
    x_imputed[pid_idx, :] = x_patient_imputed
    
print(x_imputed)

print("done")

# ---------------------- TEST DATA PREPROCESSING ---------------------------- 
# Imputation
x_test_imputed = np.zeros((int(X_test.shape[0]/12), 35))

for pid_idx in range(len(feature_index_test_unique)):
    # get training data for a patient (dim 12 x d)
    x_test_patient = get_patient_data_test(X_test,pid_idx)
    x_test_patient_imputed = np.array([0.]*35)
    
    # deal with column seperately
    for col in range(len(x_test_patient[1, :])):
        # Count number of Nan in each column
        nan_count = np.isnan(x_test_patient[:, col]).sum()
        
        if nan_count<12:
            x_test_patient_imputed[col]=np.nanmean(x_test_patient, axis = 0)[col]
        else:
            x_test_patient_imputed[col] = -1           
    # Concatenate imputed data for every patient
    x_test_imputed[pid_idx, :] = x_test_patient_imputed
    
print(x_test_imputed)

print("done")

# Once the data has been processed, save it to avoid recomputing it
np.save('x_imputed12.npy', x_imputed)
np.save('x_test_imputed12.npy', x_test_imputed)

<ipython-input-7-6058161aeff2>:14: RuntimeWarning: Mean of empty slice
  x_patient_imputed[col]=np.nanmean(x_patient, axis = 0)[col]


[[ 3.400000e+01 -1.000000e+00 -1.000000e+00 ... -1.000000e+00
   1.145000e+02  7.370000e+00]
 [ 7.100000e+01 -1.000000e+00  2.780000e+01 ...  1.000000e-02
   1.329091e+02 -1.000000e+00]
 [ 6.800000e+01 -1.000000e+00  2.090000e+01 ... -1.000000e+00
   1.170000e+02 -1.000000e+00]
 ...
 [ 5.300000e+01 -1.000000e+00 -1.000000e+00 ... -1.000000e+00
   9.925000e+01  7.402500e+00]
 [ 8.900000e+01 -1.000000e+00  3.470000e+01 ...  1.505000e+00
   1.315455e+02 -1.000000e+00]
 [ 8.500000e+01 -1.000000e+00  3.640000e+01 ... -1.000000e+00
   1.280909e+02 -1.000000e+00]]
done


<ipython-input-7-6058161aeff2>:39: RuntimeWarning: Mean of empty slice
  x_test_patient_imputed[col]=np.nanmean(x_test_patient, axis = 0)[col]


[[ 39.        -1.        41.35     ...  -1.       128.625      7.37    ]
 [ 62.        -1.        -1.       ...  -1.       111.444444  -1.      ]
 [ 72.        -1.        -1.       ...  -1.       124.909091   7.46    ]
 ...
 [ 51.        -1.        41.3      ...  -1.       101.25       7.395   ]
 [ 52.        -1.       123.1      ...  -1.       119.916667   7.4575  ]
 [ 57.        -1.        27.3      ...  -1.       119.454545   7.35    ]]
done


### task1&2

In [8]:
Labels=['pid','LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total',
         'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2',
         'LABEL_Bilirubin_direct', 'LABEL_EtCO2','LABEL_Sepsis','LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']
output = pd.DataFrame(np.array([[0]*16]), columns=Labels)

In [9]:
x_imputed = np.load('x_imputed12.npy',allow_pickle='TRUE')
x_test_imputed = np.load('x_test_imputed12.npy',allow_pickle='TRUE')

In [10]:
y_tot = y[:, 0:11]

In [11]:
# Fit the Random Forest on the whole training set
x_tot = x_imputed
y_tot = y[:, 0:11]

c=RandomForestClassifier(min_samples_leaf=20, class_weight='balanced', n_estimators=100)
# c.fit(x_tot,y_tot)

brm_clf = OneVsRestClassifier(c)
brm_clf.fit(x_tot, y_tot)

# Predict the Labels for the test features
x_test_tasksA_B = x_test_imputed
y_est_test=brm_clf.predict_proba(x_test_tasksA_B)

In [12]:
# Write in the output dataframe the obtained values
TESTS = ['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total',
         'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2',
         'LABEL_Bilirubin_direct', 'LABEL_EtCO2','LABEL_Sepsis']
i=0
for pid in feature_index_test_unique:
    output.loc[i,'pid']=pid
    output.loc[i,TESTS]=y_est_test[i,:]
    i+=1

In [13]:
# Load the imputed data, in case it has already been calculated and saves
x_imputed = np.load('x_imputed3.npy',allow_pickle='TRUE')
x_test_imputed = np.load('x_test_imputed3.npy',allow_pickle='TRUE')
print(x_imputed.shape,x_test_imputed.shape)
x_imputed = x_imputed.reshape(-1, 12*35)
x_test_imputed = x_test_imputed.reshape(-1, 12*35)
print(x_imputed.shape,x_test_imputed.shape)

(227940, 35) (151968, 35)
(18995, 420) (12664, 420)


### task3

In [14]:
def taskC_regressor(alpha_,x_imputed,y):
    c=Ridge(alpha=alpha_)

    # 10 fold cross-validation to choose hyperparameters
    num_fold = 10
    kf = KFold(n_splits=num_fold)
    score = 0
    for train_index, val_index in kf.split(x_imputed): # split the data into validation and training sets
        x_train = x_imputed[train_index]
        x_train_scaled = scaler.fit_transform(x_train)
        x_val = x_imputed[val_index]
        x_val_scaled = scaler.transform(x_val)
        
        y_train = y[train_index, 11:15]
        y_val = y[val_index, 11:15]
 
        c.fit(x_train_scaled, y_train) # apply model to training data
        y_est = np.transpose(c.predict(x_val_scaled)) # predict labels using validation data
        
        score += np.mean([0.5 + 0.5*np.maximum(0, r2_score(y_val[:,k],y_est[k,:])) for k in range(4)])
        print(np.mean([0.5 + 0.5*np.maximum(0, r2_score(y_val[:,k],y_est[k,:])) for k in range(4)]))

    # EVALUATION of model
    # Calculate average auroc score for all 10 folds
    avg_score = score/num_fold

    return avg_score



In [15]:
alpha_values = np.logspace(-3, 3, num=50, base=10.) #50 values between 10-3 to 10+3 
print (alpha_values)
avg_scores = [taskC_regressor(alpha,x_imputed,y) for alpha in alpha_values]
print(avg_scores)

# find alpha that has the highest score
best_alpha = alpha_values[np.argmax(avg_scores)]
print(best_alpha)

[1.000000e-03 1.325711e-03 1.757511e-03 2.329952e-03 3.088844e-03
 4.094915e-03 5.428675e-03 7.196857e-03 9.540955e-03 1.264855e-02
 1.676833e-02 2.222996e-02 2.947052e-02 3.906940e-02 5.179475e-02
 6.866488e-02 9.102982e-02 1.206793e-01 1.599859e-01 2.120951e-01
 2.811769e-01 3.727594e-01 4.941713e-01 6.551286e-01 8.685114e-01
 1.151395e+00 1.526418e+00 2.023590e+00 2.682696e+00 3.556480e+00
 4.714866e+00 6.250552e+00 8.286428e+00 1.098541e+01 1.456348e+01
 1.930698e+01 2.559548e+01 3.393222e+01 4.498433e+01 5.963623e+01
 7.906043e+01 1.048113e+02 1.389495e+02 1.842070e+02 2.442053e+02
 3.237458e+02 4.291934e+02 5.689866e+02 7.543120e+02 1.000000e+03]
0.7619168805955446
0.7400422821902719
0.7553550593355493
0.7635670799619407
0.7377260563986805
0.7799001193585242
0.7551645864434797
0.5
0.7549965741911598
0.7660184249289789
0.7620477300182424
0.7414661718162268
0.7561664238016785
0.7638092888117666
0.7378455519974085
0.779985843643145
0.7552544537601271
0.5
0.7560045723866651
0.7663138

0.7346588248052061
0.7321200026833412
0.7501237461445726
0.7326691222459193
0.5
0.7375824416443889
0.7449841951213646
0.7318160401684077
0.733158744959783
0.7328542580299939
0.7273331912847081
0.7258138084083441
0.741758271700355
0.7255000159581743
0.5
0.7299508601226317
0.7385766510114306
0.7233996962508089
0.7247218606030222
0.7249801474385724
0.7191227301537181
0.7183764361258553
0.7323359682717858
0.7173885475711252
0.5
0.721327545213329
0.7313911811460572
0.7140989088858043
0.7153571568261113
0.7161156748900993
0.7101052630436082
0.7099010202376415
0.7219901448024498
0.7084234103444138
0.5118259985120699
0.7118362836669976
0.7233898003251358
0.7040016089881349
0.7051709439955799
0.7063686734440302
0.7003476978944951
0.7004765894592083
0.7108622525975554
0.6986794069988388
0.5205949435782116
0.7015843669520156
0.7144737242595423
0.6931466756850188
0.6942183578960239
0.6958001177022027
0.6898690042434253
0.6901455373927083
0.6990507954945056
0.688179597200284
0.5258397542020721
0.69

In [16]:
# Fit the best Ridge regression on the whole training set
x_tot_scaled = scaler.fit_transform(x_imputed)
y_tot = y[:, 11:15]

c=Ridge(alpha=best_alpha)
c.fit(x_tot_scaled, y_tot)

x_test_scaled=scaler.transform(x_test_imputed)
y_est_test=(c.predict(x_test_scaled))

In [17]:
i=0
for pid in feature_index_test_unique:
    output.loc[i,'pid']=int(pid)
    output.loc[i,['LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']]=y_est_test[i, :]
    i+=1

In [18]:
compression_opts = dict(method='zip', archive_name='prediction.csv')
output.to_csv('prediction.zip', index=False, float_format='%.3f', compression=compression_opts)